# A common interface for handling gridded data

We've already seen that astropy has a Table class that provides a uniform way of reading, manipulating, and writing table data. Similarly, the [astropy.nddata](http://docs.astropy.org/en/stable/nddata/index.html) sub-package provides an [NDData](http://docs.astropy.org/en/stable/api/astropy.nddata.NDData.html#astropy.nddata.NDData) class that provide a common interface to datasets that consist of an n-dimensional array optionally with WCS information. In addition, this sub-package provides a set of utilities that can operate on NDData objects as well as Numpy arrays.

**Note:** This sub-package is still under development, so some aspects shown here may become easier in future.


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Create an NDData object</li>
<li>Create/read in a CCDData object</li>
<li>Extract cutouts from image data with WCS information</li>
<li>Resize n-dimensional data</li>
</ul>

</div>

</section>


## Documentation

This notebook only shows a subset of the functionality in astropy.nddata. For more information about the features presented below as well as other available features, you can read the
[astropy.nddata documentation](https://docs.astropy.org/en/stable/nddata/).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('image', origin='lower')
plt.rc('figure', figsize=(10, 6))

## Creating an NDData object

The main class in the [astropy.nddata](http://docs.astropy.org/en/stable/nddata/index.html) sub-package is [NDData](http://docs.astropy.org/en/stable/api/astropy.nddata.NDData.html#astropy.nddata.NDData):

In [ ]:
from astropy.nddata import NDData

The simplest way to initialize NDData is with a data array:

In [ ]:
import numpy as np
data = np.random.random((16, 16, 16))

In [ ]:
ndd1 = NDData(data)

In [ ]:
ndd1.data

The real value from NDData comes from the ability to add other meta-data, such as the unit, a mask, or WCS information:

In [ ]:
from astropy import units as u
from astropy.wcs import WCS
wcs = WCS(naxis=3)
mask = data > 0.5

In [ ]:
ndd2 = NDData(data, mask=mask, unit=u.Jy, wcs=wcs)

In [ ]:
ndd2.wcs

It is also possible to attach uncertainties:

In [ ]:
from astropy.nddata import StdDevUncertainty
uncertainty = StdDevUncertainty(data / 10)
ndd3 = NDData(data, uncertainty=uncertainty)

## Using the CCDData class

For now, NDData is just a data container, so it does not have useful methods on it. A more useful class is [CCDData](http://docs.astropy.org/en/stable/api/astropy.nddata.CCDData.html#astropy.nddata.CCDData) which is a sub-class of NDData and is a good example to demonstrate NDData functionality:

In [ ]:
from astropy.nddata import CCDData

The CCDData class recognizes FITS files:

In [ ]:
ccd = CCDData.read('data/gc_2mass_k.fits', unit='count')
ccd

In [ ]:
plt.subplot(projection=ccd.wcs)
plt.imshow(ccd.data, vmax=800)
plt.grid()

A useful aspect of NDData classes like CCDData is that they can be sliced like Numpy arrays and this automatically slices the mask, uncertainty, and WCS (if present):

In [ ]:
subccd = ccd[100:300, 200:400]

In [ ]:
plt.subplot(projection=subccd.wcs)
plt.imshow(subccd.data, vmax=800)
plt.grid()

Another useful feature is the ability to do arithmetic between CCDData objects and have uncertainties propagate automatically:

In [ ]:
ccd1 = CCDData([1, 2, 3], unit='count',
               uncertainty=StdDevUncertainty([1, 1.2, 1.5]))
ccd2 = CCDData([0.5, 0.5, 0.5], unit='count',
               uncertainty=StdDevUncertainty([0.2, 0.1, 0.3]))

In [ ]:
ccd_sub = ccd1.subtract(ccd2)

In [ ]:
ccd_sub.data

In [ ]:
ccd_sub.uncertainty

Note however that at the moment this framework does not understand correlated errors!

In [ ]:
ccd1.divide(ccd1).uncertainty

## Extracting cutouts from images

As seen above, we can easily extract cutouts of NDData objects by using slicing - however, the astropy.nddata sub-package also provides a [Cutout2D](http://docs.astropy.org/en/stable/api/astropy.nddata.Cutout2D.html#astropy.nddata.Cutout2D) class that can be used to generate cutouts from data, either as NDData objects or as plain arrays and WCS objects:

In [ ]:
from astropy.nddata import Cutout2D

One of the main benefits of this class is that allows cutouts to be extracted by specifying celestial coordinates rather than just pixel coordinates:

In [ ]:
from astropy.coordinates import SkyCoord
galcen = SkyCoord(0 * u.deg, 0 * u.deg, frame='galactic')

In [ ]:
cutout = Cutout2D(ccd.data, galcen, 20 * u.arcmin, wcs=ccd.wcs)

In [ ]:
plt.subplot(projection=cutout.wcs)
plt.imshow(cutout.data, vmin=300, vmax=1000) 

The Cutout2D object also retains information about where it was extracted from in the image:

In [ ]:
plt.subplot(projection=cutout.wcs)
plt.imshow(ccd.data, vmin=300, vmax=1000) 
cutout.plot_on_original(color='white') 

and we can therefore also convert positions in the cutout back to positions in the original image:

In [ ]:
cutout.to_original_position((20, 30))

## Resizing images

Another set of useful functions in astropy.nddata are [block_reduce](http://docs.astropy.org/en/stable/api/astropy.nddata.block_reduce.html#astropy.nddata.block_reduce) and [block_replicate](http://docs.astropy.org/en/stable/api/astropy.nddata.block_replicate.html#astropy.nddata.block_replicate) which can be used to downsample and upsample data by integer factors:

In [ ]:
from astropy.nddata import block_reduce, block_replicate

In [ ]:
plt.imshow(block_reduce(ccd.data, 20))

In [ ]:
plt.imshow(block_replicate(ccd.data[:10, :10], 3))

For now, this does not properly support NDData objects or scaling the WCS information (see [astropy/astropy#4796](https://github.com/astropy/astropy/issues/4796) for an example of as astropy feature request!).

<center><i>This notebook was written by <a href="https://aperiosoftware.com/">Aperio Software Ltd.</a> &copy; 2019, and is licensed under a <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License (CC BY 4.0)</a></i></center>

![cc](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by.svg)